In [ ]:
pwd

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from datetime import datetime
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import warnings
warnings.filterwarnings('ignore')

fdir = '/Users/naveenkumar/Dropbox/OnlineWork/Datasets/McKinseyData/'


In [3]:
train_data = pd.read_csv(fdir + 'train.csv')
test_data  = pd.read_csv(fdir + 'test.csv' )
full_data = [train_data, test_data]

In [53]:
def roundn(x,n):
    from math import pow
    return round(x*pow(10,n))/pow(10,n)

def columntype(x):
    t = ''
    if is_numeric_dtype(x) == True:
        t = 'numeric'
    else:
        t = 'string'
    
    return t

def unique_value_counts(x,y,cut_off):
    if y <= cut_off:
        return (s2[x].value_counts()).to_dict()
    
    
def replace_DDMMYY_WITH_DDMMYYYY(indate, AddCentury):
    if len(str(indate)) == 8:
        YY = str(indate)[6:8]
        YY_INT = int(YY) + AddCentury
        return str(indate)[0:6] + str(YY_INT)
    else:
        return np.nan
    

def datediff(d1,d2,date_format):
    df = date_format
    a = datetime.strptime(str(d1), df)
    b = datetime.strptime(str(d2), df)
    delta = b - a
    return delta.days

def percentile_dist(x):
    print("-----------------------------------")
    for t in np.arange(0.01,1.0,0.01):
        print(str(t) + " : " + str(x.quantile(q=t)))

# Calculate information value
def calc_iv(df, feature, target, pr=0):

    lst = []

    for i in range(df[feature].nunique()):
        val = list(df[feature].unique())[i]
        lst.append([feature, val, df[df[feature] == val].count()[feature], df[(df[feature] == val) & (df[target] == 1)].count()[feature]])

    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Bad'])
    data = data[data['Bad'] > 0]

    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])
    data['ValIV'] = (data['WoE'] * (data['Distribution Good'] - data['Distribution Bad']))

    data = data.sort_values(by=['Variable', 'Value'], ascending=True)

    IV = data['ValIV'].sum()
    
    if pr == 1:
        print(data)
    
    return IV        
        
def ds_summary(s2):
    cmd = pd.DataFrame(s2.columns,columns=['column_name'])
    cmd['TotalRows'] = len(s2)
    cmd['TotalUniqueLevels'] = cmd['column_name'].apply(lambda col:s2[col].nunique())
    cmd['PercentUniqueValues'] = cmd['TotalUniqueLevels']/cmd['TotalRows']
    cmd['TotalNullValues'] = cmd['column_name'].apply(lambda col:s2[col].isnull().sum())
    cmd['PercentNullValues'] = cmd['TotalNullValues']/cmd['TotalRows']
    cmd['IdColFlag'] = cmd['TotalRows'] == cmd['TotalUniqueLevels']
    cmd['DataType'] = cmd['column_name'].apply(lambda col:columntype(s2[col]))
    cmd['UniqueValue'] = cmd[cmd['TotalUniqueLevels'] < 11]['column_name'].apply(lambda col:(s2[col].value_counts()).to_dict())
    cmd['mean'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].mean(),2))
    cmd['min'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].min(),2))
    cmd['q1'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].quantile(q=0.25),2))
    cmd['median'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].median(),2))
    cmd['q3'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].quantile(q=0.75),2))
    cmd['max'] = cmd[cmd['DataType']=='numeric']['column_name'].apply(lambda col:round(s2[col].max(),2))
    return cmd


In [5]:
col_summary = ds_summary(train_data)
col_summary.to_csv(fdir + 'train_col_summary.psv', sep = "|")

In [6]:
col_list = [
 'Gender'
,'Lead_Creation_Date'
,'Monthly_Income'
,'Contacted'
,'Source_Category'
,'Var1'
,'Approved'
,'DOB'
,'Existing_EMI'
,'City_Category'
,'Employer_Category1'
,'Employer_Category2'
,'Primary_Bank_Type'
,'Loan_Amount'
,'Loan_Period'
,'Interest_Rate'
,'EMI']

In [22]:

def ApprovalRate(x):
    print('\n')
    t = train_data[[x, 'Approved']].groupby([x], as_index=False).mean()
    t['Approved'] = t['Approved']*100
    print t

ApprovalRate('Gender')
ApprovalRate('City_Category')
ApprovalRate('Employer_Category1')
ApprovalRate('Employer_Category2')
ApprovalRate('Primary_Bank_Type')
ApprovalRate('Contacted')
ApprovalRate('Source_Category')
ApprovalRate('Loan_Period')
ApprovalRate('Var1')




   Gender  Approved
0  Female  0.833221
1    Male  1.932464


  City_Category  Approved
0             A  1.655808
1             B  1.338798
2             C  0.803831


  Employer_Category1  Approved
0                  A  1.082913
1                  B  1.949490
2                  C  2.111445


   Employer_Category2  Approved
0                 1.0  2.113668
1                 2.0  2.813299
2                 3.0  1.728395
3                 4.0  1.448369


  Primary_Bank_Type  Approved
0                 G  0.946723
1                 P  2.029329


  Contacted  Approved
0         N  0.707914
1         Y  1.870790


  Source_Category  Approved
0               A  0.000000
1               B  1.660405
2               C  1.230877
3               D  0.000000
4               E  1.428571
5               F  1.307190
6               G  1.372653


   Loan_Period  Approved
0          1.0  2.226935
1          2.0  1.781528
2          3.0  2.209945
3          4.0  1.911195
4          5.0  2.035669
5     

In [8]:
col_summary

,column_name,TotalRows,TotalUniqueLevels,PercentUniqueValues,TotalNullValues,PercentNullValues,IdColFlag,DataType,UniqueValue,mean,min,q1,median,q3,max
0,ID,69713,69713,1.000000,0,0.000000,True,string,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gender,69713,2,0.000029,0,0.000000,False,string,"{u'Male': 39949, u'Female': 29764}",NaN,NaN,NaN,NaN,NaN,NaN
2,DOB,69713,10759,0.154333,15,0.000215,False,string,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Lead_Creation_Date,69713,92,0.001320,0,0.000000,False,string,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,City_Code,69713,678,0.009726,814,0.011676,False,string,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,City_Category,69713,3,0.000043,814,0.011676,False,string,"{u'A': 49885, u'C': 11694, u'B': 7320}",NaN,NaN,NaN,NaN,NaN,NaN
6,Employer_Code,69713,36617,0.525254,4018,0.057636,False,string,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Employer_Category1,69713,3,0.000043,4018,0.057636,False,string,"{u'A': 33336, u'C': 14303, u'B': 18056}",NaN,NaN,NaN,NaN,NaN,NaN
8,Employer_Category2,69713,4,0.000057,4298,0.061653,False,numeric,"{1.0: 4258, 2.0: 1955, 3.0: 1620, 4.0: 57582}",3.72,1.00,4.00,4.0,4.0,4.0
9,Monthly_Income,69713,5010,0.071866,0,0.000000,False,numeric,NaN,5622.28,0.00,1650.00,2500.0,4000.0,38383838.3


In [9]:
for tr in full_data:
    tr['LCD_NEW'] = tr['Lead_Creation_Date'].apply(lambda x:replace_DDMMYY_WITH_DDMMYYYY(x,2000))
    tr['DOB_NEW'] = tr['DOB'].apply(lambda x:replace_DDMMYY_WITH_DDMMYYYY(x,1900))
    tr["DOB_NEW"].fillna('11/01/1982', inplace=True)
    tr['LCD_DAYS'] = tr['LCD_NEW'].apply(lambda x: datediff(x,'31/01/2018',"%d/%m/%Y"))
    tr['AGE_YEAR'] = tr['DOB_NEW'].apply(lambda x: round((datediff(x,'31/01/2018',"%d/%m/%Y")*1.0)/365,0))

In [15]:
for tr in full_data:
    tr["I_Primary_Bank_Type"] = tr["Primary_Bank_Type"]
    tr["I_Primary_Bank_Type"].fillna('M', inplace=True) #New value as proportion is 13%

    tr["I_Employer_Category1"] = tr["Employer_Category1"]
    tr["I_Employer_Category1"].fillna('M', inplace=True)#New value as proportion is 6%

    tr["I_Employer_Category2"] = tr["Employer_Category2"]
    tr["I_Employer_Category2"].fillna(5, inplace=True)#New value as proportion is 6%

    tr["I_City_Category"] = tr["City_Category"]
    tr["I_City_Category"].fillna('A', inplace=True) #Existing value as proportion is 1%


In [23]:
ApprovalRate('Primary_Bank_Type')
ApprovalRate('I_Primary_Bank_Type')

ApprovalRate('Employer_Category1')
ApprovalRate('I_Employer_Category1')

ApprovalRate('Employer_Category2')
ApprovalRate('I_Employer_Category2')

ApprovalRate('City_Category')
ApprovalRate('I_City_Category')



  Primary_Bank_Type  Approved
0                 G  0.946723
1                 P  2.029329


  I_Primary_Bank_Type  Approved
0                   G  0.946723
1                   M  0.212970
2                   P  2.029329


  Employer_Category1  Approved
0                  A  1.082913
1                  B  1.949490
2                  C  2.111445


  I_Employer_Category1  Approved
0                    A  1.082913
1                    B  1.949490
2                    C  2.111445
3                    M  0.124440


   Employer_Category2  Approved
0                 1.0  2.113668
1                 2.0  2.813299
2                 3.0  1.728395
3                 4.0  1.448369


   I_Employer_Category2  Approved
0                   1.0  2.113668
1                   2.0  2.813299
2                   3.0  1.728395
3                   4.0  1.448369
4                   5.0  0.302466


  City_Category  Approved
0             A  1.655808
1             B  1.338798
2             C  0.803831


  I_City_

In [24]:
for tr in full_data:
    tr['I_Loan_Amount'] = tr['Loan_Amount']
    tr['I_Loan_Period'] = tr['Loan_Period']
    tr['I_Interest_Rate'] = tr['Interest_Rate']
    tr['I_EMI'] = tr['EMI']
    tr['I_Existing_EMI'] = tr['Existing_EMI']

    tr['I_Loan_Amount'].fillna(0, inplace = True)
    tr['I_Loan_Period'].fillna(0, inplace = True)
    tr['I_Interest_Rate'].fillna(0, inplace = True)
    tr['I_EMI'].fillna(0, inplace = True)
    tr['I_Existing_EMI'].fillna(0, inplace = True)

In [29]:
train_data['Q_Loan_Amount'] = pd.qcut(train_data['Loan_Amount'], 4)
ApprovalRate('Q_Loan_Amount')



         Q_Loan_Amount  Approved
0  (4999.999, 20000.0]  1.685313
1   (20000.0, 30000.0]  1.709623
2   (30000.0, 50000.0]  1.749105
3  (50000.0, 300000.0]  3.268299


In [34]:
train_data['Q_I_Loan_Amount'] = pd.qcut(train_data['I_Loan_Amount'], 5, duplicates='drop')
ApprovalRate('Q_I_Loan_Amount')



       Q_I_Loan_Amount  Approved
0     (-0.001, 5000.0]  0.639720
1    (5000.0, 21000.0]  1.663798
2   (21000.0, 43000.0]  1.423110
3  (43000.0, 300000.0]  2.985948


In [87]:
varlist = ['Gender','City_Category','Employer_Category1','Employer_Category2']


def calc_iv_(df, featurelist, target, roundto = 5):
    
    ivls = []
    
    for feature in featurelist:
    
        lst = []

        for i in range(df[feature].nunique()):
            val = list(df[feature].unique())[i]
            lst.append([feature, val, df[df[feature] == val].count()[feature], df[(df[feature] == val) & (df[target] == 1)].count()[feature]])

        data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Bad'])
        data = data[data['Bad'] > 0]

        data['Share'] = data['All'] / data['All'].sum()
        data['Bad Rate'] = data['Bad'] / data['All']
        data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
        data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
        data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])
        data['ValIV'] = (data['WoE'] * (data['Distribution Good'] - data['Distribution Bad']))

        data = data.sort_values(by=['Variable', 'Value'], ascending=True)

        ivls.append([feature, roundn(data['ValIV'].sum(),roundto)])
        ivds = pd.DataFrame(ivls, columns=['Variable', 'IV'])
        
    return ivds
        
calc_iv(train_data, varlist, 'Approved')

,Variable,IV
0,Gender,0.15901
1,City_Category,0.05963
2,Employer_Category1,0.09859
3,Employer_Category2,0.02707


In [79]:
len(varlist)
varlist[0]

'Gender'

In [ ]:
col_list_model = [
'Gender',
'Monthly_Income',
'Contacted',
'Source_Category',
'Var1',
'Approved',
'LCD_DAYS',
'AGE_YEAR',
'I_Primary_Bank_Type',
'I_Employer_Category1',
'I_Employer_Category2',
'I_City_Category',
'I_Loan_Amount',
'I_Loan_Period',
'I_Interest_Rate',
'I_EMI',
'I_Existing_EMI']

In [ ]:
ds = tr[col_list_model]


from sklearn import preprocessing
le = preprocessing.LabelEncoder()
ds['Contacted_dummy'] = le.fit_transform(ds['Contacted'])
ds['Gender_dummy'] = le.fit_transform(ds['Gender'])
ds['I_City_Category_dummy'] = le.fit_transform(ds['I_City_Category'])

ds['I_Primary_Bank_Type_dummy'] = le.fit_transform(ds['I_Primary_Bank_Type'])
ds['I_Employer_Category1_dummy'] = le.fit_transform(ds['I_Employer_Category1'])
ds['I_Employer_Category2_dummy'] = le.fit_transform(ds['I_Employer_Category2'])
ds['Var1_dummy'] = le.fit_transform(ds['Var1'])
ds['I_Loan_Period_dummy'] = le.fit_transform(ds['I_Loan_Period'])
ds['Source_Category_dummy'] = le.fit_transform(ds['Source_Category'])


ohe = preprocessing.OneHotEncoder()
dummy1 = pd.DataFrame(ohe.fit_transform(ds['I_City_Category_dummy'].reshape(-1,1)).toarray(), columns = ["City_CategoryA","City_CategoryB","City_CategoryC"])
dummy2 = pd.DataFrame(ohe.fit_transform(ds['I_Primary_Bank_Type_dummy'].reshape(-1,1)).toarray(), columns = ["I_Primary_Bank_Type_dummyG","I_Primary_Bank_Type_dummyM","I_Primary_Bank_Type_dummyP"])
dummy3 = pd.DataFrame(ohe.fit_transform(ds['I_Employer_Category1_dummy'].reshape(-1,1)).toarray(), columns = ["I_Employer_Category1_dummyA","I_Employer_Category1_dummyB","I_Employer_Category1_dummyC","I_Employer_Category1_dummyM"])
dummy4 = pd.DataFrame(ohe.fit_transform(ds['I_Employer_Category2_dummy'].reshape(-1,1)).toarray(), columns = ["I_Employer_Category2_dummy1","I_Employer_Category2_dummy2","I_Employer_Category2_dummy3","I_Employer_Category2_dummy4","I_Employer_Category2_dummy5"])
dummy5 = pd.DataFrame(ohe.fit_transform(ds['Var1_dummy'].reshape(-1,1)).toarray(), columns = ["Var1_dummy0","Var1_dummy2","Var1_dummy4","Var1_dummy7","Var1_dummy10"])
dummy6 = pd.DataFrame(ohe.fit_transform(ds['I_Loan_Period_dummy'].reshape(-1,1)).toarray(), columns = ["I_Loan_Period_dummy0","I_Loan_Period_dummy1","I_Loan_Period_dummy2","I_Loan_Period_dummy3","I_Loan_Period_dummy4","I_Loan_Period_dummy5","I_Loan_Period_dummy6"])
dummy7 = pd.DataFrame(ohe.fit_transform(ds['Source_Category_dummy'].reshape(-1,1)).toarray(), columns = ["Source_Category_dummyA","Source_Category_dummyB","Source_Category_dummyC","Source_Category_dummyD","Source_Category_dummyE","Source_Category_dummyF","Source_Category_dummyG"])


ds = pd.concat([ds, dummy1], axis = 1)
ds = pd.concat([ds, dummy2], axis = 1)
ds = pd.concat([ds, dummy3], axis = 1)
ds = pd.concat([ds, dummy4], axis = 1)
ds = pd.concat([ds, dummy5], axis = 1)
ds = pd.concat([ds, dummy6], axis = 1)
ds = pd.concat([ds, dummy7], axis = 1)

ds.head(10)

In [ ]:
col_summary = ds_summary(ds)
col_summary.to_csv('/Users/naveenkumar/Desktop/sklearnNotebooks/train_col_summary.psv', sep = "|")

In [ ]:
x_col = [
'Monthly_Income',
'LCD_DAYS',
'AGE_YEAR',
'I_Loan_Amount',
'I_Interest_Rate',
'I_EMI',
'I_Existing_EMI',
'Contacted_dummy',
'Gender_dummy',
'City_CategoryA',
'City_CategoryB',
#'City_CategoryC',
'I_Primary_Bank_Type_dummyG',
#'I_Primary_Bank_Type_dummyM',
'I_Primary_Bank_Type_dummyP',
'I_Employer_Category1_dummyA',
'I_Employer_Category1_dummyB',
'I_Employer_Category1_dummyC',
#'I_Employer_Category1_dummyM',
'I_Employer_Category2_dummy1',
'I_Employer_Category2_dummy2',
'I_Employer_Category2_dummy3',
'I_Employer_Category2_dummy4',
#'I_Employer_Category2_dummy5',
'Var1_dummy0',
'Var1_dummy2',
'Var1_dummy4',
'Var1_dummy7',
#'Var1_dummy10',
'I_Loan_Period_dummy0',
'I_Loan_Period_dummy1',
'I_Loan_Period_dummy2',
'I_Loan_Period_dummy3',
'I_Loan_Period_dummy4',
'I_Loan_Period_dummy5',
#'I_Loan_Period_dummy6',
'Source_Category_dummyA',
'Source_Category_dummyB',
'Source_Category_dummyC',
'Source_Category_dummyD',
'Source_Category_dummyE',
#'Source_Category_dummyF',
'Source_Category_dummyG']

y_col = 'Approved'

In [ ]:
x = ds[x_col]
y = ds[y_col]

In [ ]:
import sklearn 
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import roc_auc_score as ras 
from sklearn.metrics import precision_score as pre
from sklearn.metrics import recall_score as rec
from sklearn.metrics import f1_score as fsc
from sklearn.metrics import confusion_matrix as cfm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
def pm(y_a, y_p):
    print('acc : ' + str(acc(y_a, y_p)))
    print('pre : ' + str(pre(y_a, y_p)))
    print('rec : ' + str(rec(y_a, y_p)))
    print('fsc : ' + str(fsc(y_a, y_p)))
    print('auc : ' + str(ras(y_a, y_p)))
    print('------confusion matrix------')
    print(cfm(y_a, y_p))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size  = 0.2, random_state = 42 )

In [ ]:
for i in np.arange(1,50,1):
    rf = RandomForestClassifier(max_depth=i, random_state=42)
    rf.fit(x_train, y_train)
    pred_train = rf.predict(x_train)
    pred_test = rf.predict(x_test)
    print('AUC for MaxDepth '+ str(i) + ' tr :'+ str(round(ras(y_train, pred_train),2)) + ' ts :'+ str(round(ras(y_test, pred_test),2)))       
    #print('AUC for total tree of '+ str(i) + ' tr :'+ str(round(ras(y_train, pred_train),2)) + ' ts :'+ str(round(ras(y_test, pred_test),2)))           

In [ ]:
rf = RandomForestClassifier(max_depth=20, random_state=42)
rf.fit(x_train, y_train)
pred_train = rf.predict(x_train)
#pred_test = rf.predict(x_test)
pm(y_train, pred_train)
#pm(y_test, pred_test)

In [ ]:
for i in [500,1000]:
    lr = LogisticRegression(max_iter =i )
    lr.fit(x_train, y_train)
    pred_train = lr.predict(x_train)
    print('AUC for max_iter '+ str(i) + ' tr :'+ str(round(ras(y_train, pred_train),2)))


In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
pred_train = lr.predict(x_train)
#pred_test = lr.predict(x_test)
pm(y_train, pred_train)
#pm(y_test, pred_test)



In [ ]:
lr = LogisticRegression()
lr

In [ ]:
def t2(x):
    y = x + '_dummy'
    print('-----------------------')
    print ds.groupby([x,y]).size()
    
def t1(x):
    print('-----------------------')
    print ds[x].value_counts()

t2('Contacted')
t2('Gender')
t2('I_City_Category')
t2('I_Primary_Bank_Type')
t2('I_Employer_Category1')
t2('I_Employer_Category2')
t2('Var1')
t2('I_Loan_Period')
t2('Source_Category')

In [ ]:
gbc = GradientBoostingClassifier()
gbc.fit(x_train, y_train)
pred_train = gbc.predict(x_train)
#pred_test = gbc.predict(x_test)
pm(y_train, pred_train)
#pm(y_test, pred_test)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier(RandomForestClassifier(max_depth = 15), n_estimators=20)
abc.fit(x_train, y_train)
pred_train = abc.predict(x_train)
pred_test = gbc.predict(x_test)
pm(y_train, pred_train)
pm(y_test, pred_test)

In [ ]:
abc.feature_importances_

In [56]:
m = train_data.describe().transpose()

In [58]:
type(m)

pandas.core.frame.DataFrame

In [59]:
m.index

Index([u'Employer_Category2', u'Monthly_Income', u'Existing_EMI',
       u'Loan_Amount', u'Loan_Period', u'Interest_Rate', u'EMI', u'Var1',
       u'Approved', u'LCD_DAYS', u'AGE_YEAR', u'I_Employer_Category2',
       u'I_Loan_Amount', u'I_Loan_Period', u'I_Interest_Rate', u'I_EMI',
       u'I_Existing_EMI'],
      dtype='object')

In [62]:
m.index.name = ['Var']

In [63]:
m

,count,mean,std,min,25%,50%,75%,max
[Var],,,,,,,,
Employer_Category2,65415.0,3.720187,0.807374,1.00,4.00,4.0,4.00,4.0
Monthly_Income,69713.0,5622.283200,174767.061840,0.00,1650.00,2500.0,4000.00,38383838.3
Existing_EMI,69662.0,360.928751,2288.517927,0.00,0.00,0.0,350.00,545436.5
Loan_Amount,42004.0,39429.982859,30727.595990,5000.00,20000.00,30000.0,50000.00,300000.0
Loan_Period,42004.0,3.890629,1.167491,1.00,3.00,4.0,5.00,6.0
Interest_Rate,22276.0,19.213570,5.847136,11.99,15.25,18.0,20.00,37.0
EMI,22276.0,1101.466242,752.661394,118.00,649.00,941.0,1295.00,13556.0
Var1,69713.0,3.948446,3.819214,0.00,0.00,2.0,7.00,10.0
Approved,69713.0,0.014631,0.120073,0.00,0.00,0.0,0.00,1.0
